<a href="https://colab.research.google.com/github/tristanoprofetto/neural-networks/blob/main/GAN/DCGAN/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.models import Model
from keras.layers import Input, Reshape, Dense, Conv2D, Conv2DTranspose, UpSampling2D, Lambda, Dropout, Activation, BatchNormalization, Flatten, Layer
from keras import backend as K

In [ ]:
images = np.load("gs://quickdraw_data/apple.npy")

images = images / 255
images = np.reshape(images, (images.shape[0], 28, 28, 1))

width, height = images.shape[1:3]

FileNotFoundError: ignored

In [ ]:
def Discriminator(depth=64, dropout=0.4):

  image = Input((width, height, 1))

  # Convolutional Layers
  l1 = Conv2D(depth, kernel_size=5, strides=2, padding='same', activation='relu')(image)
  l1 = Dropout(dropout)(l1)

  l2 = Conv2D(depth * 2, kernel_size=5, strides=2, activation='relu')(l1)
  l2 = Dropout(dropout)(l2)

  l3 = Conv2D(depth * 4, kernel_size=5, strides=2, padding='same', activation='relu')(l2)
  l3 = Dropout(dropout)(l3)

  l4 = Conv2D(depth * 8, kernel_size=5, strides=2, padding='same', activation='relu')(l3)
  l4 = Flatten()(Dropout(dropout)(l4))

  output = Dense(1, activation='sigmoid', name="prediction")(l4)

  model = Model(inputs=image, outputs=output)

  return model

In [ ]:
def Generator(noise_dim=32, depth=64, dropout=0.4):

  noise = Input((noise_dim,))

  # Input Layer
  l1 = Dense(7*7*depth)(noise)
  l1 = BatchNormalization(momentum=0.9)(l1)
  l1 = Activation(activation='relu')(l1)
  l1 = Reshape((7, 7, depth))(l1)
  l1 = Dropout(dropout)(l1)

  
  # De-Convolutional Layers
  d1= UpSampling2D()(l1)
  d1 = Conv2DTranspose(int(depth / 2), kernel_size=5, padding='same', activation=None)(d1)
  d1 = BatchNormalization(momentum=0.9)(d1)
  d1 = Activation(activation='relu')(d1)

  d2= UpSampling2D()(d1)
  d2 = Conv2DTranspose(int(depth / 4), kernel_size=5, padding='same', activation=None)(d2)
  d2 = BatchNormalization(momentum=0.9)(d2)
  d2 = Activation(activation='relu')(d2)

  d3= UpSampling2D()(d2)
  d3 = Conv2DTranspose(int(depth / 8), kernel_size=5, padding='same', activation=None)(d3)
  d3 = BatchNormalization(momentum=0.9)(d3)
  d3 = Activation(activation='relu')(d3)

  output = Conv2D(1, kernel_size=5, padding='same', activation='sigmoid')(d3)

  model = Model(inputs=noise, outputs=output)

  return model

In [ ]:
disc = Discriminator()
disc.compile(loss='binary_crossentropy', optimizer=keras.optimizers.rmsprop_v2, metrics=['binary_accuracy'])

gen = Generator()

In [ ]:
# Copying the weights from the dicriminator to the 'stopped discriminator' during training
def copyWeights(source, target):
  for i, layer in enumerate(source.layers):
    target.layers[i].set_weights(source.layers[i].get_weights())

In [ ]:
def GAN():

  Z = Input(shape=(32,))

  image = gen(Z)

  pred = disc(image)

  model = Model(inputs=Z, outputs=pred)

  return model

In [ ]:
model = GAN()
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.rmsprop_v2, metrics=['accuracy'])

In [ ]:
disc_ = Discriminator()
disc_.trainable=False

In [ ]:
def trainModel(epochs=2000, batch_size=128, noise_dim=32):

  # Training metrics for discriminator and GAN
  disc_metrics = []
  disc_loss = 0
  disc_accuracy =0

  gan_metrics = []
  gan_loss = 0
  gan_accuracy =0


  for i in range(epochs):

    # Discriminator Training
    real = np.reshape(data[np.random.choice(data.shape[0], batch_size, replace=False)], (batch_size, 28, 28, 1)]) 
    fake = generator.predict(np.random.uniform(-1, 1, size=[batch_size, noise_dim]))

    X = np.concatenate((real, fake))

    y = np.ones([2 * batch_size, 1]) - np.random.uniform(0, 0.1, [2 * batch_size, 1])
    y[batch_size:,:] = 0
    y[batch_size:,:] += np.random.uniform(0,0.1,[batch_size,1])
    
    disc_metrics.append(discriminator.train_on_batch(X,y))

    disc_loss +=  disc_metrics[-1][0]
    disc_accuracy += disc+metrics[-1][1]


    # GAN Training
    noise = np.random.uniform(-1, 1, size=[batch_size, noise_dim])
    y = np.ones([batch_size, 1])

    copy_weights(discriminator, discriminator)

    gan_metrics.append(model.train_on_batch(noise, y))

    gan_loss += gan_metrics[-1][0]
    gan_accuracy += gan_metrics[-1][1]


    # Visualizing Training History
    if (i+1) % 20 == 0:
      print('Epoch {}'.format(i))


      noise = np.random.uniform(-1, 1, size=[16, noise_dim])

      gen_images = generator.predict(noise)

      plt.figure(figsize=(5,5))
      for k in range(gen_images.shape[0]):
        plt.subplot(4, 4, k+1)
        plt.imshow(gen_images[k, :, :, 0], cmap='gray')

      plt.tight_layout()
      plt.show()

  return disc_metrics, gan_metrics

    